<a href="https://colab.research.google.com/github/jane4246/CapsNet-Keras/blob/master/Copy_of_Ng'etich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing the required libraries
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.14.0
!pip install pandas_ml
!conda install pandas==0.24.2

     |████████████████████████████████| 317kB 5.3MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
     |████████████████████████████████| 377.0MB 43kB/s 
     |████████████████████████████████| 3.2MB 19.2MB/s 
     |████████████████████████████████| 491kB 38.6MB/s 
ERROR: tensorflow 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0
     |████████████████████████████████| 81kB 4.3MB/s 
  Created wheel for pandas-ml: filenam

In [2]:
import sys
#!conda install --yes --prefix {sys.prefix} pandas==0.24.2
import pandas
pandas.__version__

u'0.24.2'

In [3]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from utils import combine_images
from PIL import Image
import pandas
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
import seaborn as sns
from pandas_ml import ConfusionMatrix
sns.set()

#Required libraries being imported

#import tensorflow.keras as kerasfrom __future__ import print_function

import keras
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



import tensorflow as tf
from keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import utils
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam, SGD
from keras import optimizers
from keras.layers import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from os import listdir
import os
from imutils import paths
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
import random

K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [4]:
#Building the CapsNet
def CapsNet(input_shape, n_class, routings):
    """
    A Capsule Network on Plantvillage.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=128, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=16*n_class))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    noise = layers.Input(shape=(n_class, 16))
    noised_digitcaps = layers.Add()([digitcaps, noise])
    masked_noised_y = Mask()([noised_digitcaps, y])
    manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model, eval_model, manipulate_model


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

In [5]:
#define the model
model, eval_model, manipulate_model = CapsNet(input_shape=(28, 28, 3),
                                                  n_class=5,
                                                  routings=3)
model.summary()

W1103 08:58:08.163388 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1103 08:58:08.190578 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1103 08:58:08.194964 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1103 08:58:08.320470 139908693211008 deprecation.py:506] From capsulelayers.py:145: calling softmax (from tensorflow.python.ops.nn_ops) with dim is deprecated and will be removed in a future version.
Instructions for updating:
dim is deprecated, use axis instead


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 128)  31232       input_1[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 6, 6, 256)    2654464     conv1[0][0]                      
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 1152, 8)      0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Now rewriting the algorithm to preprocess about 500 images in my own pipeline
#only do this if you have enough RAM
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, (28, 28))
            return img_to_array(image)
        else:
            return np.array([])
        
    except Exception as e:
        print("Error : {e}")
        return None

image_list = []

directory_root = '../content/drive/MyDrive/Kahawa'
imagePaths = list(paths.list_images(directory_root))
imagePaths = imagePaths[0:-1:2]
random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-2] for p in imagePaths]

batch_size = 16
try:
    for i in np.arange(0, len(imagePaths), batch_size):
        batchPaths = imagePaths[i:i + batch_size]
        batchLabels = labels[i: i + batch_size]
        
        for image in batchPaths:
            image_list.append(convert_image_to_array(image))
except Exception as e:
    print ("Error: {e}")


len(image_list), len(labels)

np_image_list = np.array(image_list, dtype = np.float16) / 255.0
print("[INFO] splitting data")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, labels, test_size = 0.2, random_state = 42)

[INFO] splitting data


In [8]:
np_image_list.shape

(29275, 28, 28, 3)

In [9]:
y_test_unclassfied = y_test


In [10]:
y_test = LabelBinarizer().fit_transform(y_test)
y_train = LabelBinarizer().fit_transform(y_train)

In [11]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_train.shape, x_test.shape

((23420, 28, 28, 3), (5855, 28, 28, 3))

In [12]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 3))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 3))
x_train.shape, x_test.shape

((23420, 28, 28, 3), (5855, 28, 28, 3))

In [13]:
y_test.shape

(5855, 5)

In [14]:
x_train.shape, len(y_test)
classNames = [str(x) for x in np.unique(labels)]

In [15]:
#fitting the model
model.compile(optimizer=optimizers.Adam(lr=0.001),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})

weight_path = '../content/drive/MyDrive/Models for Goz project/best_weights_capsule_new_train.h5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_capsnet_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
history = model.fit([x_train, y_train], [y_train, x_train], validation_data = [[x_test, y_test], [y_test, x_test]], epochs = 50, batch_size = 16, callbacks=[checkpoint])



W1103 10:22:39.681118 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1103 10:22:39.941289 139908693211008 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1103 10:22:40.615693 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1103 10:22:40.803852 139908693211008 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.co

Train on 23420 samples, validate on 5855 samples
Epoch 1/50
23420/23420 [==============================] - 107s 5ms/step - loss: 0.1917 - capsnet_loss: 0.1805 - decoder_loss: 0.0286 - capsnet_acc: 0.7419 - val_loss: 0.1156 - val_capsnet_loss: 0.1070 - val_decoder_loss: 0.0219 - val_capsnet_acc: 0.8606

Epoch 00001: val_capsnet_acc improved from -inf to 0.86063, saving model to ../content/drive/MyDrive/Models for Goz project/best_weights_capsule_new_train.h5
Epoch 2/50
23420/23420 [==============================] - 98s 4ms/step - loss: 0.1129 - capsnet_loss: 0.1048 - decoder_loss: 0.0205 - capsnet_acc: 0.8672 - val_loss: 0.1209 - val_capsnet_loss: 0.1132 - val_decoder_loss: 0.0197 - val_capsnet_acc: 0.8509

Epoch 00002: val_capsnet_acc did not improve from 0.86063
Epoch 3/50
23420/23420 [==============================] - 101s 4ms/step - loss: 0.0837 - capsnet_loss: 0.0766 - decoder_loss: 0.0181 - capsnet_acc: 0.9134 - val_loss: 0.1029 - val_capsnet_loss: 0.0959 - val_decoder_loss: 0.017

In [ ]:
acc = model.history.history['capsnet_acc']
val_acc = model.history.history['val_capsnet_acc']
loss = model.history.history['capsnet_loss']
val_loss = model.history.history['val_capsnet_loss']
epochs = range(1, len(loss) + 1)


#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Testing Accuracy')
plt.title('Training and Testing Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.savefig('./content/drive/MyDrive/jane_leaf/code/Plots/Accuracy_Capsulenet_Colored.png')
plt.show()
plt.clf()

In [ ]:
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Testing Loss')
plt.title('Training and Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig(".../content/drive/MyDrive/jane_leaf/code/Plots/Loss_Capsulenet_Colored.png")
plt.show()


In [ ]:
#import the weights from saved
eval_model.load_weights('../content/drive/My Drive/Models for Goz project/best_weights_capsule_new_train.h5')
y_pred = []
for i in range(0, x_test.shape[0]):
    img = np.expand_dims(x_test[i], axis=0)
    prediction, y_recon = eval_model.predict(img)
    li = ["Cerscospora", "Healthy","Leaf rust","Miner", "Phoma"] 
    class_name = li[prediction.argmax()]
    y_pred.append(class_name)

In [ ]:
len(y_test_unclassfied) == len(y_pred)

True

In [ ]:
#plotting the confusion matrix and evaluating the predictions
confusion_matrix = ConfusionMatrix(y_test_unclassfied, y_pred)
print("Confusion matrix:\n%s" % confusion_matrix)

In [ ]:
confusion_matrix.print_stats()

In [ ]:
confusion_matrix.to_dataframe().to_csv('.../Results')
confusion_matrix.to_dataframe()

In [ ]:
confusion_matrix.stats_class.to_csv("./content/drive/My Drive/jane_leaf/code/Results/confusion_matrix_colored_resized.csv")
confusion_matrix.stats_class